### DenseNet-201

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
Xtrain = np.load('Xtrain.npz')
Xtrain = Xtrain['x']
Ytrain = np.load('Ytrain.npz')
Ytrain = Ytrain['y']

In [3]:
Xtest = np.load('Xtest.npz')
Xtest = Xtest['x']
Ytest = np.load('Ytest.npz')
Ytest = Ytest['y']

In [4]:
Xval = np.load('Xval.npz')
Xval = Xval['x']
Yval = np.load('Yval.npz')
Yval = Yval['y']

In [6]:
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(Xtrain.shape[1], Xtrain.shape[2], 3))

In [ ]:
Xtrain = np.expand_dims(Xtrain, axis=-1) 
Xtrain = np.repeat(Xtrain, 3, axis=-1)

In [ ]:
Xval = np.expand_dims(Xval, axis=-1)
Xval = np.repeat(Xval, 3, axis=-1)

In [ ]:
base_model.trainable = False

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Assuming num_classes is the number of classes you want to classify
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

In [ ]:
train_generator = train_datagen.flow(Xtrain, Ytrain, batch_size=32)
validation_data = (Xval, Yval)
model.fit(train_generator, epochs=10, validation_data=validation_data)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)